In [1]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from sklearn.metrics import (confusion_matrix, precision_score, auc,
                             roc_curve, recall_score, classification_report, f1_score)
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import random
import scipy.stats as stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import csv
from sklearn.cluster import KMeans
from sklearn_som.som import SOM
from sklearn import svm

# Dados NSL-KDD

In [2]:
kddtrain = pd.read_csv("kddtrain_onehot.csv").to_numpy()
kddtest = pd.read_csv("kddtest_onehot.csv").to_numpy()

# Abordagem Autoencoder-KNN

In [3]:
data = np.concatenate((kddtrain,kddtest), axis=0)
epochs = 50
batch_size = 256
optimizer = Adam(learning_rate=0.0001)
activation = "relu"
neighbors = 11

num_folds = 10

kfold = KFold(n_splits=num_folds, shuffle=True)

fold_no = 1
count = 0

#Métricas ErroRecons
matrix_ErroRecons = np.zeros((2,2))
accuracy_ErroRecons = np.zeros(num_folds)
recall_ErroRecons = np.zeros(num_folds)
precision_ErroRecons = np.zeros(num_folds)
f1_ErroRecons = np.zeros(num_folds)

#Métricas KNN
matrix_knn = np.zeros((2,2))
accuracy_knn = np.zeros(num_folds)
recall_knn = np.zeros(num_folds)
precision_knn = np.zeros(num_folds)
f1_knn = np.zeros(num_folds)

#Métricas KMeans
matrix_kmeans = np.zeros((2,2))
accuracy_kmeans = np.zeros(num_folds)
recall_kmeans = np.zeros(num_folds)
precision_kmeans = np.zeros(num_folds)
f1_kmeans = np.zeros(num_folds)

#Métricas SOM
matrix_som = np.zeros((2,2))
accuracy_som = np.zeros(num_folds)
recall_som = np.zeros(num_folds)
precision_som = np.zeros(num_folds)
f1_som = np.zeros(num_folds)

#Métricas SVM
matrix_svm = np.zeros((2,2))
accuracy_svm = np.zeros(num_folds)
recall_svm = np.zeros(num_folds)
precision_svm = np.zeros(num_folds)
f1_svm = np.zeros(num_folds)


for train, test in kfold.split(data[::, 0:data.shape[1]-3], data[::, data.shape[1]-1]):
    
    #####################################################
    ####Pré-Processamento
    
    #Separação dos Folds de Treinamento e Teste
    dataTrain = data[train]
    dataTest = data[test]
    
    #Conta Quantidade de dados
    Train = dataTrain.shape[1]
    noTrain = np.count_nonzero(dataTrain[:,Train-1] == 1)
    anTrain = np.count_nonzero(dataTrain[:, Train-1] == 0)

    #Separa dados normais e anômalos
    normalTrain = dataTrain[np.where(dataTrain[:,Train-1] == 1)]
    anomalyTrain = dataTrain[np.where(dataTrain[:,Train-1] == 0)]
    
    #Divide dados para treinamento
    porcen = 0.50
    j = int(noTrain * porcen)
    
    #Índices escolhidos aleatoriamente
    numbers_knn_normal = np.array(random.sample(range(0, normalTrain.shape[0]),j))
    numbers_knn_anomaly = np.array(random.sample(range(0,anomalyTrain.shape[0]),j))
    numbers_autoencoder_normal = np.array(list(set(np.arange(0, normalTrain.shape[0])) - set(numbers_knn_normal)))
    
    #Dados KNN
    knnTrain_normal = normalTrain[numbers_knn_normal]
    knnTrain_anomaly = anomalyTrain[numbers_knn_anomaly]
    knnTrain = np.concatenate((knnTrain_normal, knnTrain_anomaly), axis=0)
    
    #Dados Autoencoder
    autoencoderTrain = normalTrain[numbers_autoencoder_normal]
    
    #Normalização
    scaler = MinMaxScaler()
    scaler.fit(autoencoderTrain)
    autoencoderTrain = scaler.transform(autoencoderTrain)

    scaler1 = MinMaxScaler()
    scaler1.fit(knnTrain)
    knnTrain = scaler1.transform(knnTrain)

    dataTest = scaler1.transform(dataTest)
    
    ###Separação das Características e classes
    autoencoderTrain_class = autoencoderTrain[::, autoencoderTrain.shape[1]-2:autoencoderTrain.shape[1]]
    autoencoderTrain = autoencoderTrain[::, 0:autoencoderTrain.shape[1]-3]

    knnTrain_class = knnTrain[::, knnTrain.shape[1]-2:knnTrain.shape[1]]
    knnTrain = knnTrain[::, 0:knnTrain.shape[1]-3]

    modelTest_class = dataTest[::, dataTest.shape[1]-2:dataTest.shape[1]]
    modelTest = dataTest[::, 0:dataTest.shape[1]-3]
    
    #####################################################
    ####Autoencoder
    #Modela as camadas do Autoencoder
    var = autoencoderTrain.shape[1]
    input_vector = keras.Input(shape=(var,))
    en1 = layers.Dense(32, activation=activation)(input_vector)
    en2 = layers.Dense(16, activation=activation)(en1)
    de1 = layers.Dense(32, activation=activation)(en2)
    de2 = layers.Dense(var, activation=activation)(de1)

    #Gera o modelo
    autoencoder = keras.Model(input_vector, de2)

    #Compila o modelo
    autoencoder.compile(optimizer=optimizer, loss="mean_squared_error")

    #Treina o modelo
    history = autoencoder.fit(autoencoderTrain, autoencoderTrain, epochs = epochs, batch_size = batch_size, shuffle = True)
    
    #####################################################
    ####Erro de Reconstrução
    #Teste
    pred_ErroRecons = autoencoder.predict(modelTest)
    
    #Extração do erro
    mse_ErroRecons = np.mean(np.power(modelTest - pred_ErroRecons, 2), axis=1)
    error_df_ErroRecons = pd.DataFrame({'reconstruction_error': mse_ErroRecons,
                                        'true_class': modelTest_class[:,1]})
    
    #Limiar de Separação
    media = np.mean(error_df_ErroRecons['reconstruction_error'])
    desv = np.std(error_df_ErroRecons['reconstruction_error'])
    Z = 0
    teta = media + Z*desv
    
    #Separação dos erros
    predi_ErroRecons = np.zeros(modelTest.shape[0])
    error_ErroRecons = error_df_ErroRecons.to_numpy()
    predi_ErroRecons[np.where(error_ErroRecons[:,0] <= teta)] = 1
    
    #Métricas
    ab_ErroRecons = confusion_matrix(error_ErroRecons[:,1], predi_ErroRecons[:])
    accuracy_ErroRecons[count] = accuracy_score(error_ErroRecons[:,1], predi_ErroRecons[:])
    recall_ErroRecons[count] = recall_score(error_ErroRecons[:,1], predi_ErroRecons[:])
    precision_ErroRecons[count] = precision_score(error_ErroRecons[:,1], predi_ErroRecons[:])
    f1_ErroRecons[count] = f1_score(error_ErroRecons[:,1], predi_ErroRecons[:])
    matrix_ErroRecons = matrix_ErroRecons + ab_ErroRecons
    
    #####################################################
    ####KNN
    ###Treinamento KNN
    pred_knn = autoencoder.predict(knnTrain)
    mse_knn = np.mean(np.power(knnTrain - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                 'true_class': knnTrain_class[::,1]})
    error_knn = error_df_knn.to_numpy()
    neigh = KNeighborsClassifier(n_neighbors=neighbors)
    neigh.fit(error_knn[:,0].reshape(-1,1), error_knn[:,1])
    
    #Teste do Modelo
    pred_knn = autoencoder.predict(modelTest)
    mse_knn = np.mean(np.power(modelTest - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                'true_class': modelTest_class[::,1]})
    error_knn = error_df_knn.to_numpy()
    predict_knn = neigh.predict(error_knn[:,0].reshape(-1,1))
    
    #Métricas
    ab_knn = confusion_matrix(error_knn[:,1], predict_knn[:])
    accuracy_knn[count] = accuracy_score(error_knn[:,1], predict_knn[:])
    recall_knn[count] = recall_score(error_knn[:,1], predict_knn[:])
    precision_knn[count] = precision_score(error_knn[:,1], predict_knn[:])
    f1_knn[count] = f1_score(error_knn[:,1], predict_knn[:])
    matrix_knn = matrix_knn + ab_knn
    
    #####################################################
    ####KMeans
    ###Treinamento KMeans 
    pred_kmeans = autoencoder.predict(knnTrain)
    mse_kmeans = np.mean(np.power(knnTrain - pred_kmeans, 2), axis=1)
    error_df_kmeans = pd.DataFrame({'reconstruction_error': mse_kmeans,
                                     'true_class': knnTrain_class[::,1]})
    error_kmeans = error_df_kmeans.to_numpy()
    kmeans = KMeans(n_clusters=2, max_iter = 2000, random_state=0).fit(error_kmeans[:,0].reshape(-1,1))
    kmeans.labels_

    #Teste
    pred_kmeans = autoencoder.predict(modelTest)
    mse_kmeans = np.mean(np.power(modelTest - pred_kmeans, 2), axis=1)
    error_df_kmeans = pd.DataFrame({'reconstruction_error': mse_kmeans,
                                     'true_class': modelTest_class[::,1]})
    error_kmeans = error_df_kmeans.to_numpy()
    predict_kmeans = kmeans.predict(error_kmeans[:,0].reshape(-1,1))

    #Métricas
    ab_kmeans = confusion_matrix(error_kmeans[:,1], predict_kmeans[:])
    accuracy_kmeans[count] = accuracy_score(error_kmeans[:,1], predict_kmeans[:])
    recall_kmeans[count] = recall_score(error_kmeans[:,1], predict_kmeans[:])
    precision_kmeans[count] = precision_score(error_kmeans[:,1], predict_kmeans[:])
    f1_kmeans[count] = f1_score(error_kmeans[:,1], predict_kmeans[:])
    matrix_kmeans = matrix_kmeans + ab_kmeans
    
    #####################################################
    ####SOM
    ###Treinamento KMeans 
    pred_som = autoencoder.predict(knnTrain)
    mse_som = np.mean(np.power(knnTrain - pred_som, 2), axis=1)
    error_df_som = pd.DataFrame({'reconstruction_error': mse_som,
                                 'true_class': knnTrain_class[::,1]})
    error_som = error_df_som.to_numpy()
    som = SOM(m=2,n=1,dim=1)
    som.fit(error_som[:,0].reshape(-1,1))

    #Teste
    pred_som = autoencoder.predict(modelTest)
    mse_som = np.mean(np.power(modelTest - pred_som, 2), axis=1)
    error_df_som = pd.DataFrame({'reconstruction_error': mse_som,
                                 'true_class': modelTest_class[::,1]})
    error_som = error_df_som.to_numpy()
    predict_som = som.predict(error_som[:,0].reshape(-1,1))

    #Métricas
    ab_som = confusion_matrix(error_som[:,1], predict_som[:])
    accuracy_som[count] = accuracy_score(error_som[:,1], predict_som[:])
    recall_som[count] = recall_score(error_som[:,1], predict_som[:])
    precision_som[count] = precision_score(error_som[:,1], predict_som[:])
    f1_som[count] = f1_score(error_som[:,1], predict_som[:])
    matrix_som = matrix_som + ab_som
    
    #####################################################
    ####SVM
    #Treinamento SVM
    pred_svm = autoencoder.predict(knnTrain)
    mse_svm = np.mean(np.power(knnTrain - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                 'true_class': knnTrain_class[::,1]})
    error_svm = error_df_svm.to_numpy()
    clf = svm.SVC(kernel='rbf')
    clf.fit(error_svm[:,0].reshape(-1,1), error_svm[:,1])

    #Teste
    pred_svm = autoencoder.predict(modelTest)
    mse_svm = np.mean(np.power(modelTest - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                 'true_class': modelTest_class[::,1]})
    error_svm = error_df_svm.to_numpy()
    predict_svm = clf.predict(error_svm[:,0].reshape(-1,1))
    
    #Métricas
    ab_svm = confusion_matrix(error_svm[:,1], predict_svm[:])
    accuracy_svm[count] = accuracy_score(error_svm[:,1], predict_svm[:])
    recall_svm[count] = recall_score(error_svm[:,1], predict_svm[:])
    precision_svm[count] = precision_score(error_svm[:,1], predict_svm[:])
    f1_svm[count] = f1_score(error_svm[:,1], predict_svm[:])
    matrix_svm = matrix_svm + ab_svm

    
    count = count + 1
    fold_no = fold_no + 1
    
matrix_knn = matrix_knn/num_folds
matrix_kmeans = matrix_kmeans/num_folds
matrix_ErroRecons = matrix_ErroRecons/num_folds
matrix_som = matrix_som/num_folds
matrix_svm = matrix_svm/num_folds

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
136/136 [==============================] - 1s 2ms/step - loss: 0.1100
Epoch 2/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0876
Epoch 3/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0615
Epoch 4/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0447
Epoch 5/50
136/136 [==============================] - 

136/136 [==============================] - 0s 2ms/step - loss: 0.0034
Epoch 18/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0033
Epoch 19/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0032
Epoch 20/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 21/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0030
Epoch 22/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0029
Epoch 23/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0028
Epoch 24/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0028
Epoch 25/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0027
Epoch 26/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0027
Epoch 27/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0026
Epoch 28/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0026
Epoch 29/50
136/136 [=======

136/136 [==============================] - 0s 1ms/step - loss: 0.0059
Epoch 45/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0059
Epoch 46/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0059
Epoch 47/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0059
Epoch 48/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0059
Epoch 49/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0059
Epoch 50/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0059
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=

136/136 [==============================] - 1s 2ms/step - loss: 0.0600
Epoch 2/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0280
Epoch 3/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0249
Epoch 4/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0231
Epoch 5/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0118
Epoch 6/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0069
Epoch 7/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0057
Epoch 8/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0051
Epoch 9/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0046
Epoch 10/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0043
Epoch 11/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0041
Epoch 12/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0039
Epoch 13/50
136/136 [===============

136/136 [==============================] - 0s 2ms/step - loss: 0.0133
Epoch 29/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0133
Epoch 30/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0132
Epoch 31/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0132
Epoch 32/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0132
Epoch 33/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0132
Epoch 34/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0131
Epoch 35/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0131
Epoch 36/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0131
Epoch 37/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0131
Epoch 38/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0131
Epoch 39/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0131
Epoch 40/50
136/136 [=======

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
136/136 [==============================] - 1s 2ms/step -

136/136 [==============================] - 0s 1ms/step - loss: 0.0043
Epoch 12/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0041
Epoch 13/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0040
Epoch 14/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0039
Epoch 15/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0038
Epoch 16/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0037
Epoch 17/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0033
Epoch 18/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 19/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0030
Epoch 20/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0029
Epoch 21/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0028
Epoch 22/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0028
Epoch 23/50
136/136 [=======

136/136 [==============================] - 0s 2ms/step - loss: 0.0029
Epoch 39/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0029
Epoch 40/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0029
Epoch 41/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0029
Epoch 42/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0029
Epoch 43/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0029
Epoch 44/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0029
Epoch 45/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0028
Epoch 46/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0028
Epoch 47/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0028
Epoch 48/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0028
Epoch 49/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0028
Epoch 50/50
136/136 [=======

#Salva Resultados
with open('Resultados.csv', 'a', newline='') as csvfile:
    fieldnames = ['Algoritmo','Épocas','Batch','Activation','Porcentagem','Num_folds', 'Vizinhos','Acurácia','Recall','Precision','F1']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    if neighbors == 3:
        writer.writeheader()
    writer.writerow({'Algoritmo': 'KNN','Épocas': epochs,'Batch': batch_size,'Activation': activation,'Porcentagem': 100-(porcen*100),'Num_folds': num_folds, 'Vizinhos': neighbors,'Acurácia': sum(accuracy_knn)/num_folds,'Recall': sum(recall_knn)/num_folds,'Precision': sum(precision_knn)/num_folds,'F1': sum(f1_knn)/num_folds})
    writer.writerow({'Algoritmo': 'Limiar','Épocas': epochs,'Batch': batch_size,'Activation': activation,'Porcentagem': 100-(porcen*100),'Num_folds': num_folds, 'Vizinhos': '-','Acurácia': sum(accuracy_ErroRecons)/num_folds,'Recall': sum(recall_ErroRecons)/num_folds,'Precision': sum(precision_ErroRecons)/num_folds,'F1': sum(f1_ErroRecons)/num_folds})    

In [4]:
print("Autoencoder - Erro de Reconstrução:\n","Acertos: ", int(matrix_ErroRecons[0,0]+matrix_ErroRecons[1,1]), ", Erros:", int(matrix_ErroRecons[0,1]+matrix_ErroRecons[1,0]) ,"\n", matrix_ErroRecons.astype(int), "\nAcurácia: ", sum(accuracy_ErroRecons)/num_folds, "\nRecall: ", sum(recall_ErroRecons)/num_folds, "\nPrecision: ", sum(precision_ErroRecons)/num_folds, "\nF1: ", sum(f1_ErroRecons)/num_folds)

print("\nAutoencoder - KNN:\n","Acertos:", int(matrix_knn[0,0]+matrix_knn[1,1]), ", Erros:", int(matrix_knn[0,1]+matrix_knn[1,0]) ,"\n", matrix_knn.astype(int), "\nAcurácia: ", sum(accuracy_knn)/num_folds, "\nRecall: ", sum(recall_knn)/num_folds, "\nPrecision: ", sum(precision_knn)/num_folds, "\nF1: ", sum(f1_knn)/num_folds)

print("\nAutoencoder - SVM:\n","Acertos:", int(matrix_svm[0,0]+matrix_svm[1,1]), ", Erros:", int(matrix_svm[0,1]+matrix_svm[1,0]) ,"\n", matrix_svm.astype(int), "\nAcurácia: ", sum(accuracy_svm)/num_folds, "\nRecall: ", sum(recall_svm)/num_folds, "\nPrecision: ", sum(precision_svm)/num_folds, "\nF1: ", sum(f1_svm)/num_folds)

print("\nAutoencoder - KMeans:\n","Acertos:", int(matrix_kmeans[0,0]+matrix_kmeans[1,1]), ", Erros:", int(matrix_kmeans[0,1]+matrix_kmeans[1,0]) ,"\n", matrix_kmeans.astype(int), "\nAcurácia: ", sum(accuracy_kmeans)/num_folds, "\nRecall: ", sum(recall_kmeans)/num_folds, "\nPrecision: ", sum(precision_kmeans)/num_folds, "\nF1: ", sum(f1_kmeans)/num_folds)

print("\nAutoencoder - SOM:\n","Acertos:", int(matrix_som[0,0]+matrix_som[1,1]), ", Erros:", int(matrix_som[0,1]+matrix_som[1,0]) ,"\n", matrix_som.astype(int), "\nAcurácia: ", sum(accuracy_som)/num_folds, "\nRecall: ", sum(recall_som)/num_folds, "\nPrecision: ", sum(precision_som)/num_folds, "\nF1: ", sum(f1_som)/num_folds)

Autoencoder - Erro de Reconstrução:
 Acertos:  12855 , Erros: 1996 
 [[5335 1811]
 [ 185 7519]] 
Acurácia:  0.8655681629185488 
Recall:  0.9759429726086261 
Precision:  0.8109419513403295 
F1:  0.88421810542758

Autoencoder - KNN:
 Acertos: 13526 , Erros: 1324 
 [[6351  794]
 [ 530 7175]] 
Acurácia:  0.9108034499187123 
Recall:  0.9311701925503192 
Precision:  0.9004272777824006 
F1:  0.9154708397095807

Autoencoder - SVM:
 Acertos: 13271 , Erros: 1579 
 [[6098 1047]
 [ 532 7173]] 
Acurácia:  0.8936335065130218 
Recall:  0.9309280747230723 
Precision:  0.8740821661763153 
F1:  0.9008767078991197

Autoencoder - KMeans:
 Acertos: 4674 , Erros: 10177 
 [[3096 4049]
 [6127 1577]] 
Acurácia:  0.3147295517192658 
Recall:  0.20409237984544 
Precision:  0.17759260048384923 
F1:  0.18916812124764

Autoencoder - SOM:
 Acertos: 7417 , Erros: 7434 
 [[3564 3581]
 [3853 3852]] 
Acurácia:  0.4994134767689601 
Recall:  0.49863183660048493 
Precision:  0.41891762158377477 
F1:  0.45417522871331045


In [5]:
#Salva Resultados
with open('Resultados_Error.csv', 'a', newline='') as csvfile:
    fieldnames = ['Algoritmo','Métrica', 'Média']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    writer.writerow({'Algoritmo': 'ErroRecons','Métrica': accuracy_ErroRecons, 'Média': sum(accuracy_ErroRecons)/num_folds})
    writer.writerow({'Algoritmo': 'ErroRecons','Métrica': recall_ErroRecons, 'Média': sum(recall_ErroRecons)/num_folds})
    writer.writerow({'Algoritmo': 'ErroRecons','Métrica': precision_ErroRecons, 'Média': sum(precision_ErroRecons)/num_folds})
    writer.writerow({'Algoritmo': 'ErroRecons','Métrica': f1_ErroRecons, 'Média': sum(f1_ErroRecons)/num_folds})
    
    
    writer.writerow({'Algoritmo': 'KNN','Métrica': accuracy_knn, 'Média': sum(accuracy_knn)/num_folds})
    writer.writerow({'Algoritmo': 'KNN','Métrica': recall_knn, 'Média': sum(recall_knn)/num_folds})
    writer.writerow({'Algoritmo': 'KNN','Métrica': precision_knn, 'Média': sum(precision_knn)/num_folds})
    writer.writerow({'Algoritmo': 'KNN','Métrica': f1_knn, 'Média': sum(f1_knn)/num_folds})
    
    
    writer.writerow({'Algoritmo': 'SVM','Métrica': accuracy_svm, 'Média': sum(accuracy_svm)/num_folds})
    writer.writerow({'Algoritmo': 'SVM','Métrica': recall_svm, 'Média': sum(recall_svm)/num_folds})
    writer.writerow({'Algoritmo': 'SVM','Métrica': precision_svm, 'Média': sum(precision_svm)/num_folds})
    writer.writerow({'Algoritmo': 'SVM','Métrica': f1_svm, 'Média': sum(f1_svm)/num_folds})
    
    writer.writerow({'Algoritmo': 'KMeans','Métrica': accuracy_kmeans, 'Média': sum(accuracy_kmeans)/num_folds})
    writer.writerow({'Algoritmo': 'KMeans','Métrica': recall_kmeans, 'Média': sum(recall_kmeans)/num_folds})
    writer.writerow({'Algoritmo': 'KMeans','Métrica': precision_kmeans, 'Média': sum(precision_kmeans)/num_folds})
    writer.writerow({'Algoritmo': 'KMeans','Métrica': f1_kmeans, 'Média': sum(f1_kmeans)/num_folds})